In [ ]:
import os
import json
from typing import List

from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import (
    RunnablePassthrough,
    RunnableLambda,
    RunnableParallel
)

from azure.search.documents.models import VectorizedQuery
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

from dotenv import load_dotenv

load_dotenv()


True

In [ ]:
class CustomAzureSearchRetriever:
    def __init__(self, search_client, embeddings, k: int = 5):
        self.search_client = search_client
        self.embeddings = embeddings
        self.k = k

    def __call__(self, query: str):
        query_vector = self.embeddings.embed_query(query)

        vector_query = VectorizedQuery(
            vector=query_vector,
            k_nearest_neighbors=self.k,
            fields="content_vector",
            kind="vector"
        )

        results = self.search_client.search(
            search_text=query,
            vector_queries=[vector_query],
            select=[
                "id", "content", "source", "page",
                "title", "author", "chunk_id", "metadata"
            ],
            top=self.k
        )

        documents = []
        for r in results:
            metadata = {}
            if r.get("metadata"):
                try:
                    metadata = json.loads(r["metadata"])
                except:
                    pass

            metadata.update({
                "source": r.get("source", ""),
                "page": r.get("page", 0),
                "title": r.get("title", ""),
                "author": r.get("author", ""),
                "chunk_id": r.get("chunk_id", 0),
                "score": r.get("@search.score", 0),
            })

            documents.append(
                Document(
                    page_content=r.get("content", ""),
                    metadata=metadata
                )
            )

        return documents

In [12]:
def format_docs_with_citations(docs: List[Document]) -> str:
    formatted_chunks = []

    for doc in docs:
        page = doc.metadata.get("page", "N/A")
        source = os.path.basename(doc.metadata.get("source", ""))

        formatted_chunks.append(
            f"[Source: {source}, Page: {page}]\n{doc.page_content}"
        )

    return "\n\n---\n\n".join(formatted_chunks)


In [13]:
RAG_PROMPT = PromptTemplate(
    template="""
You are a knowledgeable assistant.
Use ONLY the provided context to answer the question.
Always cite page numbers in your answer.

Context:
{context}

Question:
{question}

Answer (with citations):
""",
    input_variables=["context", "question"]
)


In [14]:
def build_lcel_rag_chain(retriever: CustomAzureSearchRetriever):
    llm = ChatOpenAI(
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        model="gpt-4.1-nano",
        temperature=0.3
    )

    rag_chain = (
        RunnableParallel(
            {
                "docs": retriever,
                "question": RunnablePassthrough(),
            }
        )
        | RunnableLambda(
            lambda x: {
                "context": format_docs_with_citations(x["docs"]),
                "question": x["question"],
                "source_documents": x["docs"],
            }
        )
        | RunnableParallel(
            {
                "answer": RAG_PROMPT | llm | StrOutputParser(),
                "source_documents": lambda x: x["source_documents"],
            }
        )
    )

    return rag_chain


In [15]:
def query_rag_system(rag_chain, question: str):
    print("\n" + "=" * 80)
    print(f"❓ QUESTION: {question}")
    print("=" * 80)

    result = rag_chain.invoke(question)

    print("\n💡 ANSWER:\n")
    print(result["answer"])

    print("\n" + "=" * 80)
    print("📚 SOURCES")
    print("=" * 80)

    for i, doc in enumerate(result["source_documents"], 1):
        print(f"\n--- Source {i} ---")
        print(f"📄 Page: {doc.metadata.get('page')}")
        print(f"📝 File: {doc.metadata.get('source')}")
        print(f"📋 Title: {doc.metadata.get('title')}")
        print(f"👤 Author: {doc.metadata.get('author')}")
        print(f"🔢 Chunk ID: {doc.metadata.get('chunk_id')}")
        print(f"⭐ Score: {doc.metadata.get('score')}")
        print(f"\n📖 Content Preview:\n{doc.page_content[:300]}...")


In [21]:
def setup_rag_inference(embeddings, top_k_documents):
    search_client = SearchClient(
        endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
        index_name=os.getenv("AZURE_SEARCH_INDEX_NAME"),
        credential=AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY"))
    )

    retriever = CustomAzureSearchRetriever(
        search_client=search_client,
        embeddings=embeddings,
        k=top_k_documents
    )

    rag_chain = build_lcel_rag_chain(retriever)
    return rag_chain


In [22]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
top_k = 10

rag_chain = setup_rag_inference(embeddings, top_k)

query_rag_system(
    rag_chain,
    """Generate a detailed summary along with key findings for the research paper starting from 
    the begining till the ending. Also provide citations and references if possible."""
)



❓ QUESTION: Generate a detailed summary along with key findings for the research paper starting from 
    the begining till the ending. Also provide citations and references if possible.

💡 ANSWER:

The research paper "Engineering the RAG Stack" provides a comprehensive overview of Retrieval-Augmented Generation (RAG) systems, emphasizing their architectural components, capabilities, and design trade-offs. It begins by introducing the foundational concept of RAG, which integrates external retrieval mechanisms with generative models to enhance knowledge retrieval and response generation, surpassing traditional language models' limitations (Page 9).

The canonical RAG architecture, initially devised by Lewis et al., combines a Dense Passage Retriever (DPR) with a sequence-to-sequence generator such as BART or T5. The retriever calculates similarity scores between queries and documents using inner product measures, retrieving the top-k relevant documents from a corpus. The generator then